8개 카테고리에서 80개의 키워드 검색시 쿼터 사용량 : 67,200쿼터

키워드의 인기동영상 5개를 유튜브API를 사용해서 불러오는건 포인트 6만점 이상 사용해야됨. 그래서 불가능

내가 생각한 방법.
예시는 정치 카테고리, 키워드는 100개로 가정.

정치 카테고리의 키워드 순위
1. ㄱ
2. ㄴ
3. ㄷ
4. ㄹ
5. ㅁ
6. ㅂ
7. ㅅ
8. ㅇ
9. ㅈ
10. ㅊ
...
100. ㄷㄷㄷㄷㄷㄷ

우리가 json 파일로 저장해둔 정치 카테고리의 인기동영상을
영상 하나 하나 마다 댓글 키워드 랭킹을 매기는거임
예를들어 정치 카테고리의 1위 동영상이 A라는 동영상이라고 가정하면, A동영상의 댓글에서 키워드 랭킹을 매기는거임.
그렇게 정치 카테고리내 모든 동영상을 반복
그러면 동영상 A의 키워드 랭킹, B의 키워드 랭킹, C의 키워드 랭킹 등등 모든 동영상의 댓글을 하나하나 분석해서, 모든 영상에게 키워드 랭킹이 매겨질거임

그러면 이제 가장 인기가 많은 동영상인 A부터 차례대로, 정치 카테고리의 키워드의 순위대로 ㄱ,ㄴ,ㄷ,ㄹ,ㅁ,ㅂ,ㅅ .... 등등 비교해보는거임
그래서 만약 A동영상의 키워드 랭킹에서 "ㄱ" 키워드의 점수가 예를들어 10점 이상이면, "ㄱ" 키워드의 인기동영상 리스트에 A동영상을 넣는거임. 이렇게 키워드마다 5개씩 추출후, 다음 키워드의 인기동영상을 추출하는거임.

단점 : 반복 작업이라 시간이 얼마나 걸릴지 예상이 안감.
그래서 내가 생각한 바로는 ,
카테고리내 동영상이 50개 이상이면, 50개 까지만 동영상의 키워드 랭킹을 생성한다.
그리고 정치 카테고리의 인기 키워드 순위인  ㄱ,ㄴ,ㄷ,ㄹ... 의 인기동영상을 그 50개 안에서 추출하는거임.
만약 인기동영상이 5개까지 추출 안되더라도, 추출 된 만큼만 보여주기.

내 생각에는 50개 영상에서 앵간치 추출 될거라고 생각됨.



#카테고리내 동영상 하나하나 댓글 키워드 랭킹 추출

In [7]:
import json
from krwordrank.word import KRWordRank
from collections import defaultdict

# 파일 경로
INPUT_FILE = 'data/kiwi_nouns_video_comments.json'
OUTPUT_FILE = 'individual_keywords_ranking.json'

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 동영상별 키워드 랭킹 생성 함수
def generate_video_keyword_rankings(video_data):
    video_keyword_rankings = defaultdict(list)

    for category, videos in video_data.items():
        print(f"🔍 카테고리 '{category}' 처리 중...")

        for video in videos:
            video_id = video["video_id"]
            print(f"   🎥 동영상 {video_id} 키워드 분석 중...")

            # 동영상 댓글에서 추출된 명사 리스트 (리스트 안의 리스트를 하나의 리스트로 합침)
            all_nouns = [word for comment_nouns in video["nouns"] for word in comment_nouns]

            if len(all_nouns) < 2:  # 최소 2개 이상의 단어가 있어야 분석 가능
                print(f"   ⚠️ 동영상 {video_id}에서 추출된 명사가 부족하여 분석 불가능. 건너뜀")
                continue

            try:
                # krwordrank를 이용한 키워드 추출
                wordrank_extractor = KRWordRank(min_count=2, max_length=10, verbose=False)  
                keywords, rank, graph = wordrank_extractor.extract(all_nouns, beta=0.85, max_iter=10)

                # 키워드를 점수 기준으로 정렬하여 저장
                sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)

                video_keyword_rankings[category].append({
                    "video_id": video_id,
                    "keywords": sorted_keywords  # [(키워드, 점수), (키워드, 점수), ...]
                })

            except ValueError as e:
                if "The graph should consist of at least two nodes" in str(e):
                    print(f"   ⚠️ krwordrank 오류 발생 (노드 부족). 동영상 {video_id} 건너뜀.")
                    continue  # 오류 발생 시 해당 동영상은 건너뜀
                else:
                    raise  # 다른 오류가 발생하면 코드 중단

    return video_keyword_rankings

def main():
    # JSON 데이터 로드
    video_data = load_json(INPUT_FILE)

    # 동영상별 키워드 랭킹 생성
    video_rankings = generate_video_keyword_rankings(video_data)

    # 결과 저장
    save_json(video_rankings, OUTPUT_FILE)
    print(f"✅ 동영상 키워드 랭킹이 '{OUTPUT_FILE}'에 저장되었습니다.")

if __name__ == "__main__":
    main()


🔍 카테고리 'News & Politics' 처리 중...
   🎥 동영상 IC3iNhz02l0 키워드 분석 중...
   🎥 동영상 cvmdNi_oBYA 키워드 분석 중...
   🎥 동영상 PaFf3jmHVS8 키워드 분석 중...
   🎥 동영상 5V8bHfodNU8 키워드 분석 중...
   🎥 동영상 b3aRCbm-NhE 키워드 분석 중...
   🎥 동영상 cTlzcxEh4tE 키워드 분석 중...
   🎥 동영상 yk4Y5K94Wvk 키워드 분석 중...
   🎥 동영상 YESWdqZpgZo 키워드 분석 중...
   🎥 동영상 8eMGOnn9WsE 키워드 분석 중...
   🎥 동영상 SmMnL_rTItw 키워드 분석 중...
   🎥 동영상 VfpKABhDD9I 키워드 분석 중...
   🎥 동영상 lMB0WMHdTxY 키워드 분석 중...
   🎥 동영상 z0i2ucvT24o 키워드 분석 중...
   🎥 동영상 GmUnl6ce92Q 키워드 분석 중...
   🎥 동영상 CKmE_kccktk 키워드 분석 중...
   🎥 동영상 tXD4J563DcU 키워드 분석 중...
   🎥 동영상 UmFgLpoKOuA 키워드 분석 중...
   🎥 동영상 xlaReiFVjiQ 키워드 분석 중...
   🎥 동영상 GwJTq0DK_GE 키워드 분석 중...
   🎥 동영상 x_wR-jWg6f8 키워드 분석 중...
   🎥 동영상 e8Vchx-0NYk 키워드 분석 중...
   🎥 동영상 jv0ypTbuwX0 키워드 분석 중...
   🎥 동영상 CfK_9iDlCSo 키워드 분석 중...
   ⚠️ 동영상 CfK_9iDlCSo에서 추출된 명사가 부족하여 분석 불가능. 건너뜀
   🎥 동영상 tYffLnWjcB4 키워드 분석 중...
   🎥 동영상 aYYybP18qxg 키워드 분석 중...
   🎥 동영상 4lJVlU9OWv0 키워드 분석 중...
   🎥 동영상 7JWmSZWeObA 키워드 분석 중...
   🎥 동영상 O3MTA8jYug4 키워드 분석

인기동영상 5개 추출

In [ ]:
"""
지금은 개발 테스트 단계라, final_keywords_ranking.json의 상위 키워드 100개만 추출 해볼거야.
final_keywords_ranking.json의 상위 키워드 100개에 대해 각 키워드의 인기동영상을 최대 5개 찾아내서 유튜브 ID를 리턴 할거야. 
하나의 키워드(A라고 부를게)에 대해서, 위의 코드에서 생성된 video_keywords_ranking.json의 영상 하나하나의 키워드 랭킹에서 A키워드를 찾아. 
만약 영상의 키워드 랭킹에 A키워드가 존재하고, A키워드의 점수가 1.3점 이상이면 그 동영상의 ID는 A키워드의 인기 동영상 리스트에 삽입해줘.
인기동영상 리스트는 최대 5개 까지만 삽입하고, 5개가 되면  final_keywords_ranking.json의 다음 키워드로 넘어가게 구현해줘. 
이 로직을 반복해서 final_keyword_ranling.json의 상위 100개의 키워드의 연관 동영상을 추출하는 로직을 구현해줘 
"""

import json
from collections import defaultdict

# 파일 경로
FINAL_KEYWORDS_FILE = 'final_keywords_ranking.json'
VIDEO_KEYWORDS_FILE = "individual_keywords_ranking.json"
OUTPUT_FILE = 'related_videos_per_keyword.json'

# 키워드 최소 점수 기준
KEYWORD_SCORE_THRESHOLD = 1.3
MAX_VIDEOS_PER_KEYWORD = 5
TOP_N_KEYWORDS = 100  # 상위 100개 키워드만 처리

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 키워드별 인기 동영상 찾기
def find_top_videos_for_keywords(final_keywords, video_keyword_rankings):
    keyword_top_videos = defaultdict(list)

    for keyword, _ in final_keywords[:TOP_N_KEYWORDS]:  # 상위 100개 키워드만 처리
        print(f"🔍 키워드 '{keyword}'에 대한 인기 동영상 찾는 중...")

        for category, videos in video_keyword_rankings.items():
            for video in videos:
                video_id = video["video_id"]
                keyword_scores = dict(video["keywords"])  # 키워드 점수 딕셔너리

                # 키워드가 존재하고 점수가 1.3 이상이면 추가
                if keyword in keyword_scores and keyword_scores[keyword] >= KEYWORD_SCORE_THRESHOLD:
                    keyword_top_videos[keyword].append({
                        "video_id": video_id,
                        "score": keyword_scores[keyword]
                    })

                    # 5개 초과하면 중단
                    if len(keyword_top_videos[keyword]) >= MAX_VIDEOS_PER_KEYWORD:
                        break

            # 키워드별 최대 5개까지만 유지
            if len(keyword_top_videos[keyword]) >= MAX_VIDEOS_PER_KEYWORD:
                break

    return keyword_top_videos

def main():
    # JSON 파일 로드
    final_keywords = load_json(FINAL_KEYWORDS_FILE)  # 상위 100개 키워드
    video_keyword_rankings = load_json(VIDEO_KEYWORDS_FILE)  # 동영상별 키워드 랭킹

    # 키워드별 인기 동영상 찾기
    top_videos = find_top_videos_for_keywords(final_keywords, video_keyword_rankings)

    # 결과 저장
    save_json(top_videos, OUTPUT_FILE)
    print(f"✅ 키워드별 인기 동영상이 '{OUTPUT_FILE}'에 저장되었습니다.")

if __name__ == "__main__":
    main()
